# Bagging

In [19]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC
from sklearn.decomposition import PCA as RandomizedPCA
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import warnings
from scipy import stats 
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.linear_model import Ridge, ElasticNet, Lasso, LinearRegression, LogisticRegression
from sklearn.model_selection import LeaveOneOut, train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from scipy.stats import norm
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
import warnings
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform, uniform  
from sklearn.utils import resample
from sklearn.metrics import roc_auc_score, accuracy_score


In [3]:
df=pd.read_csv('Default.csv')
df.head()

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138950
3,No,No,529.250605,35704.493940
4,No,No,785.655883,38463.495880


In [4]:
y=df['default'] == 'Yes'
X = df[['balance', 'income']].copy()
X['student'] = (df['student'] == 'Yes').astype(int)

## AUC de Regresion logistica

In [9]:
model_lg = LogisticRegression(max_iter = 1000, random_state=42, C=1)
pipeline_lg = Pipeline([
    ('scaler', StandardScaler()),
    ('model_lg', model_lg)
])

pipeline_lg.fit(X, y)

scores_lg = cross_val_score(pipeline_lg, X, y, cv=10, scoring='accuracy')
print("Accuracy promedio:", np.mean(scores_lg))
print("Accuracy desviación estándar:", np.std(scores_lg))

scores_AUC = cross_val_score(pipeline_lg, X, y, cv=10, scoring='roc_auc')
print("AUC promedio:", np.mean(scores_AUC))
print("AUC desviación estándar:", np.std(scores_AUC))

Accuracy promedio: 0.9732
Accuracy desviación estándar: 0.0031874754901018484
AUC promedio: 0.9498768926048168
AUC desviación estándar: 0.013322168963306626


### AUC de Bagging Arbol de decision

In [ ]:
n_iter = 500
n_size = int(len(X))

modelos=[]
columnas_seleccionadas = []
pred_samples = np.zeros((n_iter, len(y)))


for i in range(n_iter):
    # bootstrap sample
    X_res, y_res = resample(X, y, replace=True, random_state=i)
    # aggregating
    X_columnas = np.random.choice(X.columns, 2, replace=False)
    X_res = X_res[X_columnas]

    arbol = DecisionTreeClassifier(random_state=42)

    arbol.fit(X_res, y_res)

    modelos.append(arbol)
    columnas_seleccionadas.append(X_columnas)


for i in range(n_iter):
    selected_cols = columnas_seleccionadas[i]
    pred_samples[i, :] = modelos[i].predict_proba(X[selected_cols])[:, 1]

bagged_pred = pred_samples.mean(axis=0)
auc_bagging = roc_auc_score(y, bagged_pred)

print("AUC del Bagging:", auc_bagging)

AUC del Bagging: 1.0
